In [21]:
import findspark
findspark.init()

In [22]:
import pyspark 
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("StudyApp").getOrCreate()


In [5]:
df=spark.read.csv("D:\\tips.csv",header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [6]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [8]:
#formating
from pyspark.ml.feature import VectorAssembler 
featureassembler=VectorAssembler(inputCols=['tip'],outputCol='tipdata')
df1=featureassembler.transform(df)
df1.show()


+----------+----+------+------+---+------+----+-------+
|total_bill| tip|   sex|smoker|day|  time|size|tipdata|
+----------+----+------+------+---+------+----+-------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2| [1.01]|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3| [1.66]|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|  [3.5]|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| [3.31]|
|     24.59|3.61|Female|    No|Sun|Dinner|   4| [3.61]|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| [4.71]|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|  [2.0]|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4| [3.12]|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2| [1.96]|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2| [3.23]|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| [1.71]|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|  [5.0]|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2| [1.57]|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|  [3.0]|
|     14.83|3.02|Female|    No|Sun|Dinner|   2| 

In [9]:
df1.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)
 |-- tipdata: vector (nullable = true)



In [12]:
fd=df1.select(['tipdata','total_bill'])
fd.show()
#final data

+-------+----------+
|tipdata|total_bill|
+-------+----------+
| [1.01]|     16.99|
| [1.66]|     10.34|
|  [3.5]|     21.01|
| [3.31]|     23.68|
| [3.61]|     24.59|
| [4.71]|     25.29|
|  [2.0]|      8.77|
| [3.12]|     26.88|
| [1.96]|     15.04|
| [3.23]|     14.78|
| [1.71]|     10.27|
|  [5.0]|     35.26|
| [1.57]|     15.42|
|  [3.0]|     18.43|
| [3.02]|     14.83|
| [3.92]|     21.58|
| [1.67]|     10.33|
| [3.71]|     16.29|
|  [3.5]|     16.97|
| [3.35]|     20.65|
+-------+----------+
only showing top 20 rows



In [13]:
#Spliting of data into tranin and test data  
from pyspark.ml.regression import LinearRegression


In [14]:
train,test=fd.randomSplit([.70,.30])

In [16]:
LR=LinearRegression(featuresCol='tipdata',labelCol='total_bill')

In [17]:
tm=LR.fit(train)

In [18]:
#coefficeint and intercept
print(tm.intercept)
print(tm.coefficients)#coefficient bet weigth and heigth#coefficeint and intercept

6.314330821886464
[4.561345315540163]


In [19]:
#evaluate the  train data
res=tm.evaluate(test)
res.predictions.show()

+-------+----------+------------------+
|tipdata|total_bill|        prediction|
+-------+----------+------------------+
|  [1.0]|      3.07|10.875676137426627|
|  [1.0]|      12.6|10.875676137426627|
| [1.17]|     32.83|11.651104841068454|
| [1.25]|      8.51|12.016012466311668|
| [1.44]|      7.74|12.882668076264299|
|  [1.5]|      8.35| 13.15634879519671|
|  [1.5]|     15.69| 13.15634879519671|
| [1.63]|     11.87|13.749323686216929|
| [1.68]|     13.42|13.977390951993938|
| [1.75]|     17.82| 14.29668512408175|
| [1.83]|     10.07|14.661592749324964|
|  [2.0]|      7.51|15.437021452966789|
|  [2.0]|     10.63|15.437021452966789|
|  [2.0]|     12.26|15.437021452966789|
|  [2.0]|     13.37|15.437021452966789|
|  [2.0]|     13.51|15.437021452966789|
|  [2.0]|     13.81|15.437021452966789|
|  [2.0]|     14.52|15.437021452966789|
|  [2.0]|     17.89|15.437021452966789|
|  [2.0]|     19.77|15.437021452966789|
+-------+----------+------------------+
only showing top 20 rows



D:\PythonAnaconda\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [20]:
print("this is R2",res.r2)#Accuracy
print("this is mean absolute error",res.meanAbsoluteError)
print("this is mean square root(RMSE)",res.meanSquaredError)


this is R2 0.30299515732829296
this is mean absolute error 4.886326742905538
this is mean square root(RMSE) 41.385758708501974
